# Configuration setup

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/aifactory/satellite_tank')
base_dir = os.getcwd()
display(base_dir)

!git clone https://github.com/ultralytics/yolov5  # clone
os.chdir('yolov5')
!pip install -r requirements.txt --quiet # install

In [ ]:
os.chdir(base_dir)
os.chdir('./yolov5')

In [ ]:
!pip install comet_ml --quiet

In [ ]:
!nvidia-smi

In [ ]:
from glob import glob
import pandas as pd

# Submission function

In [ ]:
def convert_2_submit(presubmit_nm:str, conf_thres:float=0, tag=None):
    try:
        os.makedirs('./runs/submission')
    except:
        pass

    root_path = os.path.join('./runs/presubmission', presubmit_nm)
    pred_paths = sorted(glob(os.path.join(root_path, 'labels/*.txt')))

    if len(pred_paths) == 0:
        raise ValueError('Not prediction result in the path')

    rslt_df = pd.DataFrame(
        columns=['File', 'Confidence', 'X1', 'Y1', 'X2', 'Y2', 'X3', 'Y3', 'X4', 'Y4'],
    )

    for pred_path in pred_paths:
        df = pd.read_csv(pred_path, sep=' ', header=None,)
        df.columns = ['class','xc', 'yc', 'width', 'height', 'Confidence']
        df.drop(columns='class', inplace=True)
        df['File'] = pred_path.split('/')[-1][:8]
        df['X1'] = (df.xc - 0.5*df.width)*1024
        df['Y1'] = (df.yc + 0.5*df.height)*1024
        df['X2'] = (df.xc + 0.5*df.width)*1024
        df['Y2'] = (df.yc + 0.5*df.height)*1024
        df['X3'] = (df.xc + 0.5*df.width)*1024
        df['Y3'] = (df.yc - 0.5*df.height)*1024
        df['X4'] = (df.xc - 0.5*df.width)*1024
        df['Y4'] = (df.yc - 0.5*df.height)*1024

        rslt_df = pd.concat([rslt_df, df[rslt_df.columns]], axis=0)

    if conf_thres > 0:
        rslt_df = rslt_df[rslt_df.Confidence >= conf_thres]

    display(rslt_df.sort_values('Confidence'))

    if tag is None:
        rslt_df.to_csv(
            os.path.join('./runs/submission/', presubmit_nm+'.csv')
        )
    else:
        rslt_df.to_csv(
            os.path.join('./runs/submission/', presubmit_nm + '_' + tag + '.csv')
        )

    print('Make submission csv file is done')

# Yolo weight reproduce

## yolov5m

In [ ]:
!python train.py \
--batch -1 \
--epochs 20 \
--data coco.yaml \
--weights '' \
--cfg yolov5m.yaml \
--save-period 50 \
--seed 0 \
--name coco_yolov5m_reproduce \
--bbox_interval 1

# Data v0

## yolov5m

### 8 batch baseline

In [ ]:
!python train.py \
--img 1024 \
--batch 8 \
--epochs 200 \
--data data_v0_yolov5m_8batch_baseline.yaml \
--weights yolov5m.pt \
--save-period 30 \
--seed 0 \
--name data_v0_yolov5m_8batch_baseline \
--bbox_interval 1

In [ ]:
convert_2_submit('data_v0_yolov5m_8batch_baseline', 0)

In [ ]:
!python detect.py \
--weights './runs/train/data_v0_yolov5m_8batch_baseline/weights/best.pt' \
--source ../data/oiltank_dataset/valid_images/ \
--project runs/presubmission \
--name data_v0_yolov5m_8batch_baseline \
--imgsz 1024 \
--save-txt \
--save-conf \
--save-crop \
--agnostic-nms \
--line-thickness 1 \
--augment \
--conf-thres 0.1

convert_2_submit('data_v0_yolov5m_8batch_baseline', 0)

# Data v1

## yolov5m

### 8batch baseline

In [ ]:
!python train.py \
--img 1024 \
--batch 8 \
--epochs 200 \
--data data_v1_yolov5m_8batch_baseline.yaml \
--weights yolov5m.pt \
--save-period 30 \
--seed 0 \
--name data_v1_yolov5m_8batch_baseline \
--bbox_interval 1

In [ ]:
# !python val.py \
# --weights './runs/train/baseline_yolov5l_8batch/weights/best.pt' \
# --data yolov5_baseline.yaml \
# --task val \
# --batch-size 8 \
# --imgsz 1024 \
# --save-txt

In [ ]:
!python detect.py \
--weights './runs/train/data_v1_yolov5m_8batch_baseline/weights/best.pt' \
--source ../data/oiltank_dataset/valid_images/ \
--project runs/presubmission \
--name data_v1_yolov5m_8batch_baseline \
--imgsz 1024 \
--save-txt \
--save-conf \
--save-crop \
--agnostic-nms \
--line-thickness 1 \
--augment \
--conf-thres 0.25

In [ ]:
convert_2_submit('data_v1_yolov5m_8batch_baseline_conf_thres_0.25', 0)

In [ ]:
glob('./runs/presubmission/*')

### 8 batch add option (cosine lr & multi scale)

In [ ]:
!python train.py \
--img 1024 \
--batch 8 \
--epochs 600 \
--data data_v1_yolov5m_8batch_baseline.yaml \
--weights yolov5m.pt \
--save-period 50 \
--seed 0 \
--multi-scale \
--cos-lr \
--patience 200 \
--name data_v1_yolov5m_8batch_multi_scale_lr \
--bbox_interval 1

In [ ]:
!python detect.py \
--weights './runs/train/data_v1_yolov5m_8batch_multi_scale_lr/weights/best.pt' \
--source ../data/oiltank_dataset/valid_images/ \
--project runs/presubmission \
--name data_v1_yolov5m_8batch_multi_scale_lr \
--imgsz 1024 \
--save-txt \
--save-conf \
--save-crop \
--agnostic-nms \
--line-thickness 1 \
--augment \
--conf-thres 0.1

In [ ]:
convert_2_submit('data_v1_yolov5m_8batch_multi_scale_lr', 0)

### 8 batch add option (cosine lr & multi scale) no patience

In [ ]:
!python train.py \
--img 1024 \
--batch 8 \
--epochs 600 \
--data data_v1_yolov5m_8batch_baseline.yaml \
--weights yolov5m.pt \
--save-period 50 \
--seed 0 \
--multi-scale \
--cos-lr \
--name data_v1_yolov5m_8batch_multi_scale_lr_no_patience \
--bbox_interval 1

## yolov5x

### 16 batch baseline

In [ ]:
!python train.py \
--img 1024 \
--batch 16 \
--epochs 600 \
--data data_v1_yolov5m_8batch_baseline.yaml \
--weights yolov5x.pt \
--save-period 50 \
--seed 0 \
--patience 200 \
--name data_v1_yolov5x_16batch_baseline \
--bbox_interval 1

In [ ]:
!python detect.py \
--weights './runs/train/data_v1_yolov5x_16batch_baseline/weights/best.pt' \
--source ../data/oiltank_dataset/valid_images/ \
--project runs/presubmission \
--name data_v1_yolov5x_16batch_baseline \
--imgsz 1024 \
--save-txt \
--save-conf \
--save-crop \
--agnostic-nms \
--line-thickness 1 \
--augment \
--conf-thres 0.1

In [ ]:
convert_2_submit('data_v1_yolov5x_16batch_baseline', 0)

# Data v3

## yolov5x

### imgsize 640 baseline

In [ ]:
!python train.py \
--img 640 \
--batch 32 \
--epochs 1000 \
--data data_v3.yaml \
--weights yolov5x.pt \
--save-period 200 \
--seed 0 \
--patience 200 \
--name data_v3_yolov5x_img640_32batch_baseline \
--bbox_interval 1

In [ ]:
!python detect.py \
--weights './runs/train/data_v3_yolov5x_img640_32batch_baseline/weights/best.pt' \
--source ../data/oiltank_dataset/valid_images/ \
--project runs/presubmission \
--name data_v3_yolov5x_img640_32batch_baseline \
--imgsz 640 \
--save-txt \
--save-conf \
--save-crop \
--agnostic-nms \
--line-thickness 1 \
--augment \
--conf-thres 0.1

In [ ]:
convert_2_submit('data_v3_yolov5x_img640_32batch_baseline', 0)

### 16 batch baseline

In [ ]:
!python train.py \
--img 1024 \
--batch 16 \
--epochs 1000 \
--data data_v3.yaml \
--weights yolov5x.pt \
--save-period 50 \
--seed 0 \
--patience 200 \
--name data_v3_yolov5x_16batch_baseline \
--bbox_interval 1

In [ ]:
!python detect.py \
--weights './runs/train/data_v3_yolov5x_16batch_baseline/weights/best.pt' \
--source ../data/oiltank_dataset/valid_images/ \
--project runs/presubmission \
--name data_v3_yolov5x_16batch_baseline \
--imgsz 1024 \
--save-txt \
--save-conf \
--save-crop \
--agnostic-nms \
--line-thickness 1 \
--augment \
--conf-thres 0.1

In [ ]:
convert_2_submit('data_v3_yolov5x_16batch_baseline', 0)

### 16 batch add option (cosine lr & multi scale) no patience

In [ ]:
!python train.py \
--img 1024 \
--batch 8 \
--epochs 1000 \
--data data_v3.yaml \
--weights yolov5x.pt \
--save-period 100 \
--seed 0 \
--patience 200 \
--name data_v3_yolov5x_8batch_multi_scale_lr \
--multi-scale \
--cos-lr \
--bbox_interval 1

In [ ]:
!python detect.py \
--weights './runs/train/data_v3_yolov5x_8batch_multi_scale_lr/weights/best.pt' \
--source ../data/oiltank_dataset/valid_images/ \
--project runs/presubmission \
--name data_v3_yolov5x_8batch_multi_scale_lr \
--imgsz 1024 \
--save-txt \
--save-conf \
--save-crop \
--agnostic-nms \
--line-thickness 1 \
--augment \
--conf-thres 0.1

In [ ]:
convert_2_submit('data_v3_yolov5x_8batch_multi_scale_lr', 0)